In [1]:
import kagglehub
import pandas as pd
import os
import cv2
import random
import matplotlib.pyplot as plt
import json
from tqdm import tqdm

c:\Users\stani\Documents\Code\Python\HIN\UN-number-detection\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
distribution = [('train', 0.8), ('test', 0.1), ('val', 0.1)]

In [2]:
path = kagglehub.dataset_download("stanislavlevendeev/hazmat-detection")
video_directory = os.environ["PATH_TO_DATA"]
print("Path to dataset files:", path)
print("Path to video files:", video_directory)

Path to dataset files: C:\Users\stani\.cache\kagglehub\datasets\stanislavlevendeev\hazmat-detection\versions\13
Path to video files: C:/Users/stani/Documents/WagonVideos


In [3]:
df = pd.read_csv(path + "/labels_dataframe.csv")
videos = df["Source"].unique()
videos

array(['1690279852.mp4', '1690281365.mp4', '1692830440.mp4',
       '1690801380.mp4', '1691487366.mp4', '1692875102.mp4',
       '1692945482.mp4', '1693811855.mp4', '1693954819.mp4',
       '1693805101.mp4', '1693820172.mp4', '1692787289.mp4',
       '1691496786.mp4', '1692872075.mp4', '1693308534.mp4',
       '1693308657.mp4', '1693309263.mp4', '1693820241.mp4',
       '1693820504.mp4', '1693820904.mp4', '1696009577.mp4',
       '1696374314.mp4', '1696416413.mp4', '1696441496.mp4'], dtype=object)

In [4]:
available_videos = os.listdir(video_directory)
available_videos = [video for video in available_videos if video.endswith(".mp4")]
available_videos = [video for video in available_videos if video in videos]
available_videos

['1690281365.mp4', '1692830440.mp4', '1696009577.mp4']

In [5]:
total_frames = df[df["Source"].isin(available_videos)]
total_frames = total_frames["Absolute Frame"].count()
total_frames

np.int64(867)

In [6]:
frames_path = "./data/frames_videos"

In [16]:
def save_frame(frame, video, frame_number, path, overwrite=True):
    """
    Save a frame to disk with a formatted filename.

    Args:
        frame (numpy.ndarray): The frame to save.
        video (str): The name of the video file (without extension).
        frame_number (int): The frame number.
        path (str): The directory to save the frame.
        overwrite (bool): Whether to overwrite existing files.
    """
    os.makedirs(path, exist_ok=True)
    formatted_frame_number = f"{frame_number:05d}"
    image_path = f"{path}/{video}_{formatted_frame_number}.jpg"
    # print(f"🎞️ Saving frame {image_path}...")
    if not overwrite and os.path.exists(image_path):
        return
    cv2.imwrite(image_path, frame)


coco_categories = [
    {
        "id": 1,
        "name": "hazmat",
    }
]
coco_images = []
coco_annotations = []


def add_annotation(video, frame_number, video_h, video_w, annotations):
    global coco_annotations
    global coco_images
    global coco_categories
    image_id = len(coco_images) + 1
    coco_images.append(
        {
            "id": image_id,
            "file_name": f"{video}_{frame_number:05d}.jpg",
            "width": video_w,
            "height": video_h,
        }
    )
    for index, annotation in annotations.iterrows():
        x_left_top = annotation["XTL"]
        y_left_top = annotation["YTL"]
        x_right_bottom = annotation["XBR"]
        y_right_bottom = annotation["YBR"]
        # get the bounding box
        width = x_right_bottom - x_left_top
        height = y_right_bottom - y_left_top
        # calculate the area
        area = width * height
        # make the bbox
        bbox = [x_left_top, y_left_top, width, height]
        coco_annotations.append(
            {
                "id": len(coco_annotations) + 1,
                "image_id": image_id,
                "category_id": 1,
                "bbox": bbox,
                "area": area,
                "iscrowd": 0,
            }
        )

In [17]:
with tqdm(total=total_frames, desc="Processing") as pbar:
    for video in available_videos:
        video_path = f"{video_directory}/{video}"
        cap = cv2.VideoCapture(video_path)
        video_w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        video_h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        number_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_number = 0
        video_name = video.split('.')[0]
        while frame_number < number_frames:
            ret, frame = cap.read()
            pbar.set_description(f"Processing {video}, frame {frame_number}/{number_frames}")
            if not ret:
                break
            annotations = df[(df["Source"] == video) & (df["Relative Frame"] == frame_number)]
            if not annotations.empty:
                # take rnd distribution if not full and then store in it.
                save_frame(frame, video_name, frame_number, frames_path)
                add_annotation(video_name, frame_number, video_h, video_w, annotations)
                pbar.update(annotations.shape[0])
            frame_number += 1
        cap.release()

Processing 1696009577.mp4, frame 4639/4640: : 950it [05:12,  3.04it/s]                        


In [12]:
coco_data = {
    "images": coco_images,
    "annotations": coco_annotations,
    "categories": coco_categories,
}
# store
with open("data/coco_data.json", "w") as f:
    json.dump(coco_data, f)

In [20]:
# check the validity of created annotations 
coco_data    

{'images': [{'id': 1,
   'file_name': '1690281365.mp4_00000.jpg',
   'width': 1920,
   'height': 1080},
  {'id': 2,
   'file_name': '1690281365.mp4_00001.jpg',
   'width': 1920,
   'height': 1080},
  {'id': 3,
   'file_name': '1690281365.mp4_00002.jpg',
   'width': 1920,
   'height': 1080},
  {'id': 4,
   'file_name': '1690281365.mp4_00003.jpg',
   'width': 1920,
   'height': 1080},
  {'id': 5,
   'file_name': '1690281365.mp4_00004.jpg',
   'width': 1920,
   'height': 1080},
  {'id': 6,
   'file_name': '1690281365.mp4_00051.jpg',
   'width': 1920,
   'height': 1080},
  {'id': 7,
   'file_name': '1690281365.mp4_00052.jpg',
   'width': 1920,
   'height': 1080},
  {'id': 8,
   'file_name': '1690281365.mp4_00053.jpg',
   'width': 1920,
   'height': 1080},
  {'id': 9,
   'file_name': '1690281365.mp4_00054.jpg',
   'width': 1920,
   'height': 1080},
  {'id': 10,
   'file_name': '1690281365.mp4_00055.jpg',
   'width': 1920,
   'height': 1080},
  {'id': 11,
   'file_name': '1690281365.mp4_0005

In [18]:
def draw_annotations(annotation):
    # coco_images is array of dictionaries with keys: id, file_name, width, height
    image_name = [image["file_name"] for image in coco_images if image["id"] == annotation["image_id"]]
    # open the image
    image = cv2.imread(f"{frames_path}/{image_name[0]}")
    # get the bbox
    bbox = annotation["bbox"]
    # draw the bbox
    x, y, w, h = bbox
    cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 2)
    # show the image
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.show()
    
def get_random_annotation():
    return random.choice(coco_annotations)

In [19]:
rnd_annotation = get_random_annotation()
print(rnd_annotation)
draw_annotations(rnd_annotation)

{'id': 38, 'image_id': 38, 'category_id': 1, 'bbox': [1677.67, 395.07, 139.52999999999997, 54.129999999999995], 'area': 7552.758899999998, 'iscrowd': 0}


error: OpenCV(4.10.0) :-1: error: (-5:Bad argument) in function 'rectangle'
> Overload resolution failed:
>  - Can't parse 'pt1'. Sequence item with index 0 has a wrong type
>  - Can't parse 'pt1'. Sequence item with index 0 has a wrong type
>  - Can't parse 'rec'. Expected sequence length 4, got 2
>  - Can't parse 'rec'. Expected sequence length 4, got 2
